<a href="https://colab.research.google.com/github/habibarezq/ML-Assignments-25/blob/main/Assignment-2/notebooks/assignment-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,DataLoader ## to feed the data into the model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

# 1. Data Preparation

### Required Preprocessing and Transformations

In [57]:
transform=transforms.Compose([
    transforms.ToTensor()
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomResizedCrop(224)
])

Load Training and test data

In [58]:
dataset=datasets.MNIST(root="./data", download=True, train=True, transform=transform)
test_data=datasets.MNIST(root="./data", download=True, train=False, transform=transform)


Extract data and labels

In [59]:
X= dataset.data.float() /255.0
y=dataset.targets

print(f"Full training data shape: {X.shape}")
print(f"Full training labels shape: {y.shape}")
print(f"Pixel value range: [{X.min():.2f}, {X.max():.2f}]")

Full training data shape: torch.Size([60000, 28, 28])
Full training labels shape: torch.Size([60000])
Pixel value range: [0.00, 1.00]


### Data Filtering

In [61]:
# Filter first: only digits 0 and 1 from the whole dataset
mask_train = (y == 0) | (y == 1)
X_binary = X[mask_train]
y_binary = y[mask_train]

### Data Splitting



In [62]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_binary, y_binary, test_size=0.4, stratify=y_binary, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

### Flatten Images

In [64]:
X_train_flat = X_train.reshape(-1, 28*28)
X_val_flat = X_val.reshape(-1, 28*28)
X_test_flat = X_test.reshape(-1, 28*28)

### Create Dataloaders

In [ ]:
train_loader = DataLoader(
    TensorDataset(X_train_flat, y_train.float()),
    batch_size=32,
    shuffle=True
)
val_loader = DataLoader(
    TensorDataset(X_val_flat, y_val.float()),
    batch_size=32,
    shuffle=False
)
test_loader = DataLoader(
    TensorDataset(X_test_flat, y_test.float()),
    batch_size=32,
    shuffle=False
)

## Logisitic Regression Implementation

In [68]:
def sigmoid(x):
  return 1 / (1+torch.exp(-x))

def binary_cross_entropy(y_pred,y_true):
  eps=1e-8 # avoid log(0)
  return -torch.mean(y_true *  torch.log(y_pred + eps) + (1-y_true)*torch.log(1-y_pred))

def compute_accuracy(y_pred,y_true):
  pass

def update_weights(X_batch,y_batch,W,b,alpha):
      """
    Perform one gradient descent update for logistic regression.

    Parameters:
        X_batch (Tensor): Batch of input features, shape (batch_size, n_features)
        y_batch (Tensor): Batch of true labels, shape (batch_size,)
        W (Tensor): Weight matrix
        b (Tensor): Bias term
        alpha (float): Learning rate

      """
      # Forward Pass
      scores= X_batch @ W +b
      y_pred=sigmoid(scores)

      # Compute loss y_pred --> (batch_size,1) while y_batch has (batch,) --> use unsqueeze to a dimension
      loss=binary_cross_entropy(y_pred,y_batch.unsqueeze(1))

      #Compute gradients
      error = y_pred - y_batch.unsqueeze(1)
      dW=(X_batch.T @ error ) / X_batch.shape[0]
      db=error.mean() # summation of error for batch divided by their number

      #Update Weights
      W -= alpha * dW
      b -= alpha * db

      return W,b

### Initalize weights and bias


In [ ]:
n_features=28*28
W=torch.zeros(n_features,1)
b=torch.zeros(1)

## Learning rate
alpha=0.01
epochs=100

### Training Loop

In [ ]:
train_losses,val_losses=[]
train_accuracies,val_accuracies=[]

for epoch in range(epochs):
